In [26]:
# python standard libraries
import os
import random
import fnmatch
import datetime
import pickle
import numpy as np
import pandas as pd
# sklearn
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

# imaging
import cv2
from imgaug import augmenters as img_aug
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
from PIL import Image

import torch
from torch.utils.data import Dataset, DataLoader
import random
import numpy as np
import cv2
import torch.nn as nn
import torch.optim as optim


In [27]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #  torch.cuda.is_available() GPU를 사용가능하면 True, 아니라면 False를 리턴
print("지금 사용하는 device :",device)

지금 사용하는 device : cuda:0


In [28]:
# 이미지 데이터가 저장된 디렉터리 경로 설정
data_dir = './data/images'

# 해당 디렉터리 내의 모든 파일 이름을 리스트로 가져옴
file_list = os.listdir(data_dir)

# 이미지 경로와 조향 각도(steering angle)를 저장할 리스트 초기화
image_paths = []
steering_angles = []

# .png 파일 패턴을 찾기 위한 설정
pattern = "*.png"

# 파일 리스트를 순회하며 각 파일에 대해 작업 수행
for filename in file_list:
    # 파일 이름이 .png로 끝나는지 확인
    if fnmatch.fnmatch(filename, pattern):
        # 이미지 경로를 리스트에 추가
        image_paths.append(os.path.join(data_dir, filename))
        
        # 파일 이름에서 조향 각도를 추출하여 리스트에 추가
        # 예: "video01_143_092.png"의 마지막 3자리 부분을 추출하여 조향 각도로 사용
        angle = int(filename[-7:-4])  # 092 부분이 조향 각도를 나타냄 (90은 직진을 의미)
        steering_angles.append(angle)


In [29]:
train_data, test_data, train_label, test_label = train_test_split(image_paths, steering_angles, test_size=0.2)
print("Training data: %d\nValidation data: %d" % (len(train_data), len(test_data)))

Training data: 175
Validation data: 44


In [30]:
def my_imread(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

def zoom(image):
    zoom = img_aug.Affine(scale=(1, 1.3))  # zoom from 100% (no zoom) to 130%
    image = zoom.augment_image(image)
    return image

def pan(image):
    # pan left / right / up / down about 10%
    pan = img_aug.Affine(translate_percent= {"x" : (-0.1, 0.1), "y": (-0.1, 0.1)})
    image = pan.augment_image(image)
    return image

def adjust_brightness(image):
    # increase or decrease brightness by 30%
    brightness = img_aug.Multiply((0.7, 1.3))
    image = brightness.augment_image(image)
    return image

def blur(image):
    kernel_size = random.randint(1, 5)  # kernel larger than 5 would make the image way too blurry
    image = cv2.blur(image,(kernel_size, kernel_size))
   
    return image

def random_flip(image, steering_angle):
    is_flip = random.randint(0, 1)
    if is_flip == 1:
        # randomly flip horizon
        image = cv2.flip(image,1)
        steering_angle = 180 - steering_angle
   
    return image, steering_angle

def random_augment(image, steering_angle):
    """
    무작위로 이미지 증강을 수행하여 다양한 주행 환경을 시뮬레이션하는 함수.

    매번 호출 시 확률적으로 여러 증강 기법(팬, 확대/축소, 블러, 밝기 조정 등)을 적용.
    이미지가 좌우로 반전될 경우 조향각도 반전.

    매개변수:
    - image: 입력 이미지
    - steering_angle: 원본 이미지에 대한 조향각

    반환값:
    - 증강된 이미지와 조정된 조향각
    """
    if np.random.rand() < 0.5:  # 50% 확률로 팬(pan) 효과 적용
        image = pan(image)
    if np.random.rand() < 0.5:  # 50% 확률로 확대/축소(zoom) 효과 적용
        image = zoom(image)
    #if np.random.rand() < 0.5:  # 50% 확률로 블러(blur) 효과 적용
    #    image = blur(image)
    if np.random.rand() < 0.5:  # 50% 확률로 밝기 조정 효과 적용
         image = adjust_brightness(image)

    # 50% 확률로 이미지 좌우 반전 및 조향각 조정
    image, steering_angle = random_flip(image, steering_angle)
    
    return image, steering_angle

def img_preprocess(image):
    """
    입력 이미지를 전처리하여 Nvidia 모델에 적합한 형식으로 변환.

    매개변수:
    - image: 입력 이미지 (RGB 형식)

    반환값:
    - 전처리된 이미지
    """
    height, _, _ = image.shape
    # 1. 이미지 상단 절반 제거 (차선 추적에 불필요한 정보 제거)
    image = image[int(height/2):, :, :]
    
    # 2. YUV 색 공간으로 변환 (Nvidia 모델에서 권장)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
    
    # 3. 가우시안 블러링 (노이즈 감소)
    image = cv2.GaussianBlur(image, (3, 3), 0)
    
    # 4. 이미지 크기 조정 (200x66 크기로 변경)
    image = cv2.resize(image, (200, 66))
    
    # 5. 정규화 (픽셀 값을 [0, 1] 범위로 변환)
    image = image / 255.0  # 모델 입력에 적합한 스케일로 변환

    return image

In [31]:

# 사용자 정의 데이터셋 클래스 정의
class CustomImageDataset(Dataset):
    def __init__(self, image_paths, steering_angles, is_training=True):
        """
        이미지 데이터와 조향각 데이터를 PyTorch Dataset 형식으로 정의하는 클래스.

        매개변수:
        - image_paths: 이미지 경로 리스트
        - steering_angles: 각 이미지에 해당하는 조향각 리스트
        - is_training: 학습 여부 (True: 학습, False: 테스트)
        """
        self.image_paths = image_paths
        self.steering_angles = steering_angles
        self.is_training = is_training

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        # 이미지와 조향각 불러오기
        image_path = self.image_paths[idx]
        image = my_imread(image_path)
        steering_angle = self.steering_angles[idx]

        # 학습 중일 경우 데이터 증강 수행
        if self.is_training:
            image, steering_angle = random_augment(image, steering_angle)

        # 이미지 전처리 수행
        image = img_preprocess(image)

        # 채널 순서 변경 (H, W, C) -> (C, H, W) for PyTorch
        image = np.transpose(image, (2, 0, 1))

        return torch.tensor(image, dtype=torch.float32), torch.tensor(steering_angle, dtype=torch.float32)

In [32]:
# 학습 데이터셋 및 데이터로더 생성
train_dataset = CustomImageDataset(train_data, train_label, is_training=True)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

# 학습 데이터셋 및 데이터로더 생성
test_dataset = CustomImageDataset(test_data, test_label, is_training=False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

In [33]:
a, b = next(iter(train_loader))

In [34]:
a[0].shape

torch.Size([3, 66, 200])

In [35]:
b

tensor([107.])

In [36]:
# 첫 번째 채널만 선택하여 그레이스케일로 표시
plt.imshow(a[1][0, :, :], cmap='viridis')
plt.colorbar()
plt.title("Single Channel as Grayscale")
plt.axis('off')
plt.show()


IndexError: index 1 is out of bounds for dimension 0 with size 1

In [ ]:

# # 배치 단위로 데이터 로딩 및 시각화 예시
# X_train, y_train = image_paths, steering_angles  # 학습용 이미지 경로 및 조향각 리스트
# X_valid, y_valid = image_paths, steering_angles  # 검증용 이미지 경로 및 조향각 리스트

# # 배치 단위로 데이터 로딩 예시
# for batch_idx, (images, angles) in enumerate(test_loader):
#     # 시각화에 사용할 행(row)과 열(column)의 수 설정
#     ncol = 2  # 열 개수 (학습 데이터, 검증 데이터)
#     nrow = min(2, len(images))  # 행 개수 (표시할 배치의 크기)

#     # 학습 데이터 배치 생성 (데이터 증강 포함)
#     X_train_batch, y_train_batch = images, angles

#     # 플롯 생성
#     fig, axes = plt.subplots(nrow, ncol, figsize=(15, 6))
#     fig.tight_layout()  # 플롯 간 간격 조정

#     # 배치 크기만큼 반복하여 각 이미지를 시각화
#     for i in range(nrow):
#         # 학습 데이터 시각화 (첫 번째 열)
#         axes[i][0].imshow(X_train_batch[i].permute(1, 2, 0).numpy())
#         axes[i][0].set_title(f"Training, angle={y_train_batch[i]:.2f}")  # 학습 데이터 조향각 표시
#         axes[i][0].axis('off')  # 축 숨김

#     # 플롯 표시
#     plt.show()


In [37]:

class NvidiaModel(nn.Module):
    def __init__(self):
        super(NvidiaModel, self).__init__()
        
        # Convolutional Layers
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 24, kernel_size=5, stride=2),  # input channels = 3 (RGB)
            nn.ELU(),
            nn.Conv2d(24, 36, kernel_size=5, stride=2),
            nn.ELU(),
            nn.Conv2d(36, 48, kernel_size=5, stride=2),
            nn.ELU(),
            nn.Conv2d(48, 64, kernel_size=3),
            nn.ELU(),
            nn.Dropout(0.2),  # Dropout layer for robustness
            nn.Conv2d(64, 64, kernel_size=3),
            nn.ELU()
        )
        
        # Fully Connected Layers
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.2),  # Dropout layer for robustness
            nn.Linear(64 * 1 * 18, 100),  # flattened output size of conv layers
            nn.ELU(),
            nn.Linear(100, 50),
            nn.ELU(),
            nn.Linear(50, 10),
            nn.ELU(),
            nn.Linear(10, 1)  # Output layer for the steering angle
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

# Model, loss function, and optimizer
model = NvidiaModel().to(device)

# Mean Squared Error loss, as this is a regression problem
criterion = nn.MSELoss()

# Adam optimizer with learning rate 1e-3
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [38]:
import tqdm
num_epochs = 300
count = 0
loss_list = []
iteration_list = []
accuracy_list = []

predictions_list = []
labels_list = []

for epoch in tqdm.tqdm(range(num_epochs)): # 총 5번만 돌린다.
    for images, labels in train_loader: # 데이터셋이 60,000개 있으니까 배치로는 100개씩 꺼내 쓰니 1에폭마다 600번 for문이 돌아간다. count는 5 * 600 = 3000이 되어야 함
        images, labels = images.to(device), labels.to(device)

        outputs = model(images).squeeze() 
        loss = criterion(outputs,labels) 
        optimizer.zero_grad() 
        loss.backward() 
        optimizer.step() 
        count += 1

        if not(count % 50): # 50의 간격으로 실제 추론해본다
            total = 0
            correct = 0
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                labels_list.append(labels)

                outputs = model(images).squeeze() 
                accuracy = criterion(outputs,labels) 

            loss_list.append(loss.data) # 지금의 손실 함수를 기록
            iteration_list.append(count) # 횟수 기록
            accuracy_list.append(accuracy) # 정확도 기록

        if not (count % 500): # 매 500회마다 값들 출력
            print("Iteration: {}, Loss: {}, Accuracy: {}%".format(count,loss.data,accuracy)) 

torch.save(model.state_dict(), "./model_state_dict.pt") # 모델 저장


  0%|          | 0/300 [00:00<?, ?it/s]d:\Program_Files\anaconda3\envs\Deep_Learning\Lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
  1%|          | 3/300 [00:05<09:46,  1.97s/it]

Iteration: 500, Loss: 299.6730041503906, Accuracy: 1238.421142578125%


  2%|▏         | 5/300 [00:09<09:40,  1.97s/it]

Iteration: 1000, Loss: 639.4818725585938, Accuracy: 3405.029541015625%


  3%|▎         | 8/300 [00:15<09:37,  1.98s/it]

...

Iteration: 51500, Loss: 641.1182250976562, Accuracy: 872.056884765625%


 99%|█████████▉| 297/300 [09:38<00:06,  2.09s/it]

Iteration: 52000, Loss: 725.4459838867188, Accuracy: 99.74640655517578%


100%|██████████| 300/300 [09:45<00:00,  1.95s/it]

Iteration: 52500, Loss: 2294.491943359375, Accuracy: 88.4798355102539%


In [65]:
for images, labels in test_loader:
    images, labels = images.to(device), labels.to(device)

    outputs = model(images).squeeze() 
    print(labels)
    print(outputs)
    loss = criterion(outputs,labels) 
    print(loss)
    break

tensor([69.], device='cuda:0')
tensor(82.5678, device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor(184.0857, device='cuda:0', grad_fn=<MseLossBackward0>)
